In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score
sns.set()

In [19]:
users=pd.read_csv('takehome_users.csv',encoding='latin-1')
engagement=pd.read_csv('takehome_user_engagement.csv',encoding='latin-1')

In [20]:
users.head()


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [21]:
engagement.head()


,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [22]:
engagement['time_stamp']=pd.to_datetime(engagement['time_stamp'],infer_datetime_format=True)
engagement.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [23]:
print(type(engagement['time_stamp'][0]))


<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [24]:
temp=pd.pivot_table(engagement, values='visited', index=['user_id', 'time_stamp'])
temp.head()


visited
user_id time_stamp                  
1       2014-04-22 03:53:30        1
2       2013-11-15 03:45:04        1
        2013-11-29 03:45:04        1
        2013-12-09 03:45:04        1
        2013-12-25 03:45:04        1

In [25]:
level_values = temp.index.get_level_values
temp_2 = (temp.groupby([level_values(i) for i in [0,1]] +[pd.Grouper(freq='1w', level=-1)])).count()

temp_2=temp_2.reset_index(level=1)
temp_2=temp_2.drop(['time_stamp'],axis=1)
temp_2=temp_2.reset_index()

temp_3=temp_2.groupby(['user_id','time_stamp']).sum()
temp_3=temp_3.reset_index()
temp_3=temp_3.groupby('user_id').max()
final=temp_3.drop('time_stamp',axis=1)
final=final.reset_index()

In [26]:
final.head()


,user_id,visited
0,1,1
1,2,3
2,3,1
3,4,1
4,5,1


In [27]:
total_df=users.merge(final, how='left',  left_on='object_id', right_on='user_id')
total_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,user_id,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1.0,1.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2.0,3.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,3.0,1.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,4.0,1.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,5.0,1.0


In [28]:
total_df=total_df.drop(['user_id'],axis=1)


In [29]:
total_df.head()


,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,3.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,1.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,1.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,1.0


In [30]:
total_df['adopted']=total_df.visited>=3
total_df.head()



,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,visited,adopted
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,1.0,False
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,3.0,True
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,1.0,False
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,1.0,False
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,1.0,False


In [31]:
total_df=total_df.drop(['object_id','creation_time','name','email','last_session_creation_time','visited','invited_by_user_id'],axis=1)
total_df.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted
0,GUEST_INVITE,1,0,11,False
1,ORG_INVITE,0,0,1,True
2,ORG_INVITE,0,0,94,False
3,GUEST_INVITE,0,0,1,False
4,GUEST_INVITE,0,0,193,False


In [32]:
total_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 5 columns):
creation_source               12000 non-null object
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
adopted                       12000 non-null bool
dtypes: bool(1), int64(3), object(1)
memory usage: 480.5+ KB


In [33]:
total_df=total_df.dropna()


In [34]:
total_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 5 columns):
creation_source               12000 non-null object
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
adopted                       12000 non-null bool
dtypes: bool(1), int64(3), object(1)
memory usage: 480.5+ KB


In [35]:
y=total_df['adopted'].values
df_x=total_df.drop(['adopted'],axis=1)
df_x=pd.get_dummies(df_x,drop_first=False)
X=df_x.values
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42,stratify=y)

In [36]:
logreg=LogisticRegression()
param_grid = {'C':[0.01,0.1,1,10,100]}
logreg_cv = GridSearchCV(logreg,param_grid,cv=5)

logreg_cv.fit(X_train,y_train)
predicted=logreg_cv.predict(X_test)
score=logreg_cv.score(X_train,y_train)
print(score)

/Users/lu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/Users/lu/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#lo

0.8795238095238095


In [37]:
logreg.fit(X_train,y_train)
coef=logreg.coef_
columns=df_x.columns

df_coef=pd.DataFrame(data=coef[0],index=columns)
df_ranked=df_coef.sort_values(0)
df_ranked

,0
creation_source_PERSONAL_PROJECTS,-0.897103
creation_source_ORG_INVITE,-0.390273
creation_source_SIGNUP,-0.252289
creation_source_SIGNUP_GOOGLE_AUTH,-0.146247
creation_source_GUEST_INVITE,-0.141253
org_id,0.001405
opted_in_to_mailing_list,0.041102
enabled_for_marketing_drip,0.060363


In [38]:
total_df.head()


,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted
0,GUEST_INVITE,1,0,11,False
1,ORG_INVITE,0,0,1,True
2,ORG_INVITE,0,0,94,False
3,GUEST_INVITE,0,0,1,False
4,GUEST_INVITE,0,0,193,False


In [39]:
creation=total_df.groupby('creation_source').mean()
creation

,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted
creation_source,,,,
GUEST_INVITE,0.250116,0.147480,141.343504,0.150717
ORG_INVITE,0.253409,0.151387,138.940762,0.115421
PERSONAL_PROJECTS,0.242539,0.143534,144.294173,0.069635
SIGNUP,0.251557,0.152851,146.248203,0.131768
SIGNUP_GOOGLE_AUTH,0.244043,0.149458,141.523466,0.148736


In [40]:
opted=total_df.groupby('opted_in_to_mailing_list').mean()
opted

,enabled_for_marketing_drip,org_id,adopted
opted_in_to_mailing_list,,,
0,0.049967,141.639129,0.118477
1,0.448230,142.622912,0.126253


In [41]:
enabled=total_df.groupby('enabled_for_marketing_drip').mean()
enabled

#creation_source has the strongest impact on user adopotion

,opted_in_to_mailing_list,org_id,adopted
enabled_for_marketing_drip,,,
0,0.161834,141.402527,0.119612
1,0.748884,144.630580,0.125000
